In [1]:
from lasagne_project_utils import load_data, shared_dataset
import theano
from __future__ import print_function
import sys
import os
import time
import numpy as np
import theano
import theano.tensor as T
import lasagne
import scipy

Using gpu device 0: GRID K520 (CNMeM is disabled, cuDNN not available)
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [56]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.eval().shape[0] == targets.eval().shape[0]
    if shuffle:
        indices = np.arange(inputs.eval().shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.eval().shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [5]:
datasets = load_data()

True
/home/ubuntu/data/cifar-10-matlab.tar.gz


In [39]:

rng = np.random.RandomState(23455)

X_train, y_train = datasets[0]
X_val, y_val = datasets[1]
X_test, y_test = datasets[2]
#X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

n_train_batches = X_train.get_value(borrow=True).shape[0]
n_valid_batches = X_val.get_value(borrow=True).shape[0]
n_test_batches = X_test.get_value(borrow=True).shape[0]
n_train_batches //= batch_size
n_valid_batches //= batch_size
n_test_batches //= batch_size


x = T.tensor4('x')
y = T.ivector('y')

channel = 3
imsize= 32

In [42]:
num_epochs = 2

X_train = X_train.reshape((9000,3,32,32))
X_test = X_test.reshape((10000, 3, 32, 32))
X_val = X_val.reshape((1000, 3, 32, 32))

In [43]:
network = lasagne.layers.InputLayer(shape=(None, 3, imsize, imsize), stride=(1,1), pad=1,input_var=x)
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=96, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(),
            pad=1,
            stride=(1,1))

print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=96, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            pad=1,
            stride=(1,1))

print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=96, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            pad=1, 
            stride=(2,2))

print(lasagne.layers.get_output_shape(network))


network = lasagne.layers.Conv2DLayer(
            network, num_filters=192, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            pad=1,
            stride=(1,1))
print(lasagne.layers.get_output_shape(network))


network = lasagne.layers.Conv2DLayer(
            network, num_filters=192, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            pad=1,
            stride=(1,1))
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=192, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            pad=1, 
            stride=(2,2))
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=192, filter_size=(3, 3),
            nonlinearity=lasagne.nonlinearities.rectify,
            stride=(1,1))

print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=192, filter_size=(1, 1),
            nonlinearity=lasagne.nonlinearities.rectify,
            stride=(1,1))
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Conv2DLayer(
            network, num_filters=10, filter_size=(1, 1),
            nonlinearity=lasagne.nonlinearities.rectify,
            stride=(1,1))
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.Pool2DLayer(network, pool_size=(6, 6), mode='average_inc_pad')
print(lasagne.layers.get_output_shape(network))

network = lasagne.layers.DenseLayer(
            lasagne.layers.dropout(network, p=.5),
            num_units=10,
            nonlinearity=lasagne.nonlinearities.softmax)
print(lasagne.layers.get_output_shape(network))


(None, 3, 32, 32)
(None, 96, 32, 32)
(None, 96, 32, 32)
(None, 96, 16, 16)
(None, 192, 16, 16)
(None, 192, 16, 16)
(None, 192, 8, 8)
(None, 192, 6, 6)
(None, 192, 6, 6)
(None, 10, 6, 6)
(None, 10, 1, 1)
(None, 10)


array([6, 9, 9, 4, 1], dtype=int32)

In [44]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, y)
loss = loss.mean()
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.01, momentum=0.9)

# Create a loss expression for validation/testing. The crucial difference
# here is that we do a deterministic forward pass through the network,
# disabling dropout layers.
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction, y)



In [45]:
test_loss = test_loss.mean()
# As a bonus, also create an expression for the classification accuracy:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), y),
                  dtype=theano.config.floatX)

# Compile a function performing a training step on a mini-batch (by giving
# the updates dictionary) and returning the corresponding training loss:
train_fn = theano.function([x, y], loss, updates=updates)

# Compile a second function computing the validation loss and accuracy:
val_fn = theano.function([x, y], [test_loss, test_acc])

In [57]:

# We could add some weight decay as well here, see lasagne.regularization.

# Create update expressions for training, i.e., how to modify the
# parameters at each training step. Here, we'll use Stochastic Gradient
# Descent (SGD) with Nesterov momentum, but Lasagne offers plenty more.

# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs:
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 500, shuffle=True):
        inputs, targets = batch
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

# After training, we compute and print the test error:
test_err = 0
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500, shuffle=False):
    inputs, targets = batch
    err, acc = val_fn(inputs, targets)
    test_err += err
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test loss:\t\t\t{:.6f}".format(test_err / test_batches))
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

Starting training...


TypeError: ('Bad input argument to theano function with name "<ipython-input-45-2929167d2310>:8"  at index 0(0-based)', 'Expected an array-like object, but found a Variable: maybe you are trying to call a function on a (possibly shared) variable instead of a numeric array?')